<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.2.1 
# *Querying the International Space Station*

## The OpenNotify API

The OpenNotify API exposes a few attributes of the International Space Station (ISS) via a simple, authentication-free interface. The simplicity of this API precludes any need for a dedicated Python library. However, as with many APIs, it accepts requests according to HTTP standards and returns responses in JSON format, so the Python libraries request and json will make managing the I/O simpler still.

In [2]:
import requests
import json
from datetime import datetime, date, time

This request fetches the latest position of the international space station:

In [ ]:
response = requests.get("http://api.open-notify.org/iss-now.json")

Print the status code and text of the response:

In [ ]:
response.status_code

200

In [ ]:
response.text


'{"timestamp": 1670012945, "message": "success", "iss_position": {"longitude": "-134.0643", "latitude": "27.0429"}}'

We can use another API to request the current position of the ISS and the next few times at which it will be over a certain location. The latitude and longitude of Sydney are (-33.87, 151.21).

In [ ]:
response = requests.get("https://api.g7vrd.co.uk/v1/satellite-passes/25544/-33.87/151.21.json?minelevation=0&hours=24")

Print the response header:

In [ ]:
response.headers


{'Date': 'Fri, 02 Dec 2022 20:33:35 GMT', 'Server': 'Apache', 'Vary': 'Origin,Access-Control-Request-Method,Access-Control-Request-Headers', 'Access-Control-Allow-Origin': '*', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Cache-Control': 'no-cache, no-store, max-age=0, must-revalidate', 'Pragma': 'no-cache', 'Expires': '0', 'X-Frame-Options': 'DENY', 'Content-Type': 'application/json', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Transfer-Encoding': 'chunked'}

Print the content of the response (the data that the server returned):

In [ ]:
response.content


b'{"api_status":"ALPHA","request_timestamp":"2022-12-02T20:33:35.699031702Z","norad_id":25544,"satellite_name":"ISS","tle_last_retrieved":"2022-12-02T12:07:35.496179522Z","lat":-33.87,"lon":151.21,"hours":24,"min_elevation":0,"query_ms":13,"passes":[{"start":"2022-12-02T21:35:35.686Z","tca":"2022-12-02T21:41:05.686Z","end":"2022-12-02T21:46:40.686Z","aos_azimuth":226,"los_azimuth":46,"max_elevation":84.0},{"start":"2022-12-02T23:13:30.686Z","tca":"2022-12-02T23:17:30.686Z","end":"2022-12-02T23:21:10.686Z","aos_azimuth":258,"los_azimuth":348,"max_elevation":7.0},{"start":"2022-12-03T12:39:35.686Z","tca":"2022-12-03T12:44:35.686Z","end":"2022-12-03T12:49:30.686Z","aos_azimuth":347,"los_azimuth":116,"max_elevation":18.0},{"start":"2022-12-03T14:15:45.686Z","tca":"2022-12-03T14:21:15.686Z","end":"2022-12-03T14:26:30.686Z","aos_azimuth":294,"los_azimuth":140,"max_elevation":34.0},{"start":"2022-12-03T15:54:45.686Z","tca":"2022-12-03T15:58:45.686Z","end":"2022-12-03T16:02:45.686Z","aos_azimu

Note that this is a Python byte string:

In [ ]:
print(type(response.content))

<class 'bytes'>


Print just the "content-type" value from the header:

In [ ]:
response.headers["Content-Type"]

'application/json'

JSON was designed to be easy for computers to read, not for people. The `requests` library can decode the JSON byte string:

In [ ]:
overheads = response.json()
print(overheads)

{'api_status': 'ALPHA', 'request_timestamp': '2022-12-02T20:33:35.699031702Z', 'norad_id': 25544, 'satellite_name': 'ISS', 'tle_last_retrieved': '2022-12-02T12:07:35.496179522Z', 'lat': -33.87, 'lon': 151.21, 'hours': 24, 'min_elevation': 0, 'query_ms': 13, 'passes': [{'start': '2022-12-02T21:35:35.686Z', 'tca': '2022-12-02T21:41:05.686Z', 'end': '2022-12-02T21:46:40.686Z', 'aos_azimuth': 226, 'los_azimuth': 46, 'max_elevation': 84.0}, {'start': '2022-12-02T23:13:30.686Z', 'tca': '2022-12-02T23:17:30.686Z', 'end': '2022-12-02T23:21:10.686Z', 'aos_azimuth': 258, 'los_azimuth': 348, 'max_elevation': 7.0}, {'start': '2022-12-03T12:39:35.686Z', 'tca': '2022-12-03T12:44:35.686Z', 'end': '2022-12-03T12:49:30.686Z', 'aos_azimuth': 347, 'los_azimuth': 116, 'max_elevation': 18.0}, {'start': '2022-12-03T14:15:45.686Z', 'tca': '2022-12-03T14:21:15.686Z', 'end': '2022-12-03T14:26:30.686Z', 'aos_azimuth': 294, 'los_azimuth': 140, 'max_elevation': 34.0}, {'start': '2022-12-03T15:54:45.686Z', 'tca': 

What kind of object did this give us?

In [ ]:
print(type(overheads))


<class 'dict'>


Python dicts are easier to work with, but the data we want is still buried in that data structure, so we have to dig it out. First, extract the `passes` value to a separate list:

In [ ]:
passes = overheads['passes']
passes

[{'start': '2022-12-02T21:35:35.686Z',
  'tca': '2022-12-02T21:41:05.686Z',
  'end': '2022-12-02T21:46:40.686Z',
  'aos_azimuth': 226,
  'los_azimuth': 46,
  'max_elevation': 84.0},
 {'start': '2022-12-02T23:13:30.686Z',
  'tca': '2022-12-02T23:17:30.686Z',
  'end': '2022-12-02T23:21:10.686Z',
  'aos_azimuth': 258,
  'los_azimuth': 348,
  'max_elevation': 7.0},
 {'start': '2022-12-03T12:39:35.686Z',
  'tca': '2022-12-03T12:44:35.686Z',
  'end': '2022-12-03T12:49:30.686Z',
  'aos_azimuth': 347,
  'los_azimuth': 116,
  'max_elevation': 18.0},
 {'start': '2022-12-03T14:15:45.686Z',
  'tca': '2022-12-03T14:21:15.686Z',
  'end': '2022-12-03T14:26:30.686Z',
  'aos_azimuth': 294,
  'los_azimuth': 140,
  'max_elevation': 34.0},
 {'start': '2022-12-03T15:54:45.686Z',
  'tca': '2022-12-03T15:58:45.686Z',
  'end': '2022-12-03T16:02:45.686Z',
  'aos_azimuth': 246,
  'los_azimuth': 152,
  'max_elevation': 7.0},
 {'start': '2022-12-03T17:34:20.686Z',
  'tca': '2022-12-03T17:37:20.686Z',
  'end': '20

Now extract the `start` strings into an array called `srisetimes`:

In [ ]:
srisetimes = []

for i in passes:
  srisetimes.append(i['start'])

srisetimes  

['2022-12-02T21:35:35.686Z',
 '2022-12-02T23:13:30.686Z',
 '2022-12-03T12:39:35.686Z',
 '2022-12-03T14:15:45.686Z',
 '2022-12-03T15:54:45.686Z',
 '2022-12-03T17:34:20.686Z',
 '2022-12-03T19:11:20.686Z',
 '2022-12-03T20:47:35.686Z']

These are strings. We convert these to an array of Python `datetime` values called `risetimes`:

In [ ]:
risetimes = [datetime.strptime(xpass['start'], "%Y-%m-%dT%H:%M:%S.%fZ") for xpass in passes]
risetimes

[datetime.datetime(2022, 12, 2, 21, 35, 35, 686000),
 datetime.datetime(2022, 12, 2, 23, 13, 30, 686000),
 datetime.datetime(2022, 12, 3, 12, 39, 35, 686000),
 datetime.datetime(2022, 12, 3, 14, 15, 45, 686000),
 datetime.datetime(2022, 12, 3, 15, 54, 45, 686000),
 datetime.datetime(2022, 12, 3, 17, 34, 20, 686000),
 datetime.datetime(2022, 12, 3, 19, 11, 20, 686000),
 datetime.datetime(2022, 12, 3, 20, 47, 35, 686000)]

Finally, use `risetime.strftime` to print these in a format that people understand:

```
e.g.
18/10/22 07:05
18/10/22 08:41
18/10/22 10:20
18/10/22 12:00
18/10/22 01:37
18/10/22 03:13
```



In [ ]:
for risetime in risetimes:
    print(risetime.strftime('%d/%m/%y %I:%M'))

02/12/22 09:35
02/12/22 11:13
03/12/22 12:39
03/12/22 02:15
03/12/22 03:54
03/12/22 05:34
03/12/22 07:11
03/12/22 08:47


Finally, here is an endpoint that tells us who is on board:

In [ ]:
response = requests.get("http://api.open-notify.org/astros.json")

Referring to the methods used above, extract the number of astronauts and their names:

In [ ]:
astros = response.json()
print(astros)
print(astros["number"])
[print(astronaut['name']) for astronaut in astros['people']]

{'people': [{'craft': 'Tiangong', 'name': 'Cai Xuzhe'}, {'craft': 'Tiangong', 'name': 'Chen Dong'}, {'craft': 'Tiangong', 'name': 'Liu Yang'}, {'craft': 'ISS', 'name': 'Sergey Prokopyev'}, {'craft': 'ISS', 'name': 'Dmitry Petelin'}, {'craft': 'ISS', 'name': 'Frank Rubio'}, {'craft': 'ISS', 'name': 'Nicole Mann'}, {'craft': 'ISS', 'name': 'Josh Cassada'}, {'craft': 'ISS', 'name': 'Koichi Wakata'}, {'craft': 'ISS', 'name': 'Anna Kikina'}, {'craft': 'Shenzhou 15', 'name': 'Fei Junlong'}, {'craft': 'Shenzhou 15', 'name': 'Deng Qingming'}, {'craft': 'Shenzhou 15', 'name': 'Zhang Lu'}], 'number': 13, 'message': 'success'}
13
Cai Xuzhe
Chen Dong
Liu Yang
Sergey Prokopyev
Dmitry Petelin
Frank Rubio
Nicole Mann
Josh Cassada
Koichi Wakata
Anna Kikina
Fei Junlong
Deng Qingming
Zhang Lu


[None, None, None, None, None, None, None, None, None, None, None, None, None]

## HOMEWORK


1. Write a simple handler for the response status code (refer to lab resources slide for HTTP response codes). As this Jupyter Notebook is an interactive device, the handler does not need to manage subsequent code execution (i.e. by branching or aborting execution), although it should return something that could be used to do so if deployed in a Python program.

In [10]:
#ANSWER:
def handleResponse(response, verbose = False):
    '''
    Returns Boolean Value, Status Code, 
    '''
    response_codes = {
        '1': 'Informational',
        '3': 'redirection',
        '4': 'Client Error',
        '5': 'Server Error'
    }

    if response.status_code == 200:
        if verbose:
            print('Response = 200: OK')
        fail = False
    else:

        if verbose:
            print('Response = ' + str(response.status_code) + ': ' + f'{response_codes[str(response.status_code)[0]]}')
        fail = True
    return(fail, response.status_code)

  # if Status Code is 200 return false, and status code
  # Otherwise Return True and Status Code

2. Test your response handler on some correct and incorrect API calls.

In [11]:
response = requests.get("http://api.open-notify.org/astros.json")
if handleResponse(response)[0]:
    print('API call failed. Resolve issue before continuing!')
    
response = requests.get("http://api.open-notify.org/iss-now.json")
handleResponse(response, True)[0]

Response = 200: OK


False

In [13]:
response = requests.get("http://api.open-notify.org/iss-now.jsonss")
handleResponse(response, True)

Response = 404: Client Error


(True, 404)

>

>

>



---



---



> > > > > > > > > © 2022 Institute of Data


---



---



